In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


def forage_rule(dmin, dmax, c, f, maxt, maxc):
    """Determine the optimal strategy.
    
    From the fitness attributed to each condition at the end of the day, this 
    function determines the optimal strategy of individuals (either forage or 
    rest), by following the principle of dynamic optimization. Fitness obtained
    by both strategies in each condition are calculated backwards in time and
    compared to retain the strategy leading to the best fitness.
    
    Parameters
    ----------
    dmin: float (between 0 and 1)
        probability of death per time unit if you are very lean.
    dmax: float (between 0 and 1)
        probability of death per time unit if you are very heavy.
    c: float (between 0 and 1)
        rate of consuming resources (= probability to decrease of one condition
                                     when resting).
    f: float (between 0 and 1)
        feeding efficiency (= probability to increase of one condition when
                            foraging).
    maxt: int > 0
        maximum time (= number of time units the day is divided into).
    maxc: int > 0
        maximum condition (= number of different condition units).

    Reminder: rows indicate conditions, columns indicate time.
    Rows are chosen like this: condition 0 (dead individuals) = row 0,
    condition 1 = row 1, condition 2 = row 2, condition 3 = row 3, ..., 
    condition maxc (individuals in top condition) = row maxc.
    Columns are chosen like this: time unit 1 = column 0,
    time unit 2 = column 1, time unit 3 = column 2, ...,
    time unit maxt = column maxt - 1.

    Return
    -------
    The output is the forage_rule matrix, with 1 denoting foraging, and 0
    denoting resting.
    """
    reward = np.zeros([maxc + 1, maxt + 1])  # Building of the initial matrix
                                             # with maxc + 1 to include the row
                                             # of dead individuals, and
                                             # maxt + 1 to include the reward
                                             # column at the end of the day.
    # We fix terminal rewards, which increase with condition:
    reward[:, maxt] = np.array([elt for elt in range(0, maxc + 1)])
    d = np.linspace(dmin, dmax, maxc)  # Death probability increases linearly
                                       # with individual condition.
    # Any individual alive can improve its condition, stay or die depending on
    # its strategy with the following probabilities:
    p_eat_up = (1 - d) * f
    p_eat_same = (1 - d) * (1 - f)
    p_eat_dead = d
    p_rest_same = 1 - c
    p_rest_down = c
    # Except individuals in top condition that cannot improve their condition
    # when foraging:
    ptop_eat_same = 1 - d[-1]
    ptop_eat_dead = d[-1]
    
    reward_if_forage = np.zeros([maxc + 1, maxt + 1])  # Building of the
                                                       # fitness matrix when
                                                       # foraging.
    reward_if_rest = np.zeros([maxc + 1, maxt + 1])  # Building of the fitness
                                                     # matrix when resting.
    forage_rule = np.zeros([maxc + 1, maxt + 1])  # Building of the optimal
                                                  # strategy matrix.
    # We start from the last time unit of the day and continue backwards to 
    # calculate fitness in each condition for both strategies:
    for t in range(maxt - 1, -1, -1):  # We start at column maxt - 1 as we 
                                       # already fixed fitness at the end of
                                       # the day in column maxt.
        for i in range(1, maxc):  # We omit special cases (rows 0 and maxc).
            reward_if_forage[i, t] = p_eat_same[i] * reward[i, t + 1] \
            + p_eat_up[i] * reward[i + 1, t + 1] + p_eat_dead[i] * 0
            reward_if_rest[i, t] = p_rest_same * reward[i, t + 1] \
            + p_rest_down * reward[i - 1, t + 1]
        # Special case 1 (row 0: dead individuals):
        reward_if_forage[0, t] = 0  # Dead individuals do not get reward at all
        reward_if_rest[0, t] = 0    # and cannot improve their condition.
        # Special case 2 (row maxc: individuals in top condition):
        reward_if_forage[maxc, t] = ptop_eat_same * reward[maxc, t + 1] \
        + ptop_eat_dead * 0  # Individuals in top condition cannot improve
                             # their condition.
        reward_if_rest[maxc, t] = p_rest_same * reward[maxc, t + 1] \
        + p_rest_down * reward[maxc - 1, t + 1]
        
        # Now we can determine what is the best strategy for each condition of
        # the time unit t by comparing the fitness matrixes, with 1 for
        # foraging and 0 for resting:
        forage_rule[:, t] = reward_if_forage[:, t] > reward_if_rest[:, t]
        # The initial matrix is completed for the time unit t with the fitness
        # value of the best strategy:
        reward[:, t] = multiply_matrix_list(forage_rule[:, t], 
                                            reward_if_forage[:, t]) \
                       + multiply_matrix_list((1 - forage_rule[:, t]), 
                                              reward_if_rest[:, t])           
    return forage_rule


def one_day_simulation(dmin, dmax, c, f, maxt, maxc, numbers):
    """Fill the matrix with individuals.
    
    From a given number of individuals in each conditions at the first time 
    unit of the day, this function calculates the number of individuals in each 
    condition at the next time units, following the best strategies according 
    to the forage_rule matrix and depending on the parameters given.
    
    Parameters
    ----------
    dmin: float (between 0 and 1)
        probability of death per time unit if you are very lean.
    dmax: float (between 0 and 1)
        probability of death per time unit if you are very heavy.
    c: float (between 0 and 1)
        rate of consuming resources (= probability to decrease of one condition
                                     when resting).
    f: float (between 0 and 1)
        feeding efficiency (= probability to increase of one condition when
                            foraging).
    maxt: int > 0
        maximum time (number of time units the day is divided into).
    maxc: int > 0
        maximum condition (number of different condition units).
    numbers: array, np.shape(numbers) = (maxc + 1, maxt + 1)
        matrix completed with the number of individuals in each condition for
        the first column

    Return
    -------
    The output is the numbers matrix recording the number of individuals in
    each condition for each time unit of the day. The total number of
    individuals at the end of the day can not correspond to the total number
    of individuals at the beginning of the day due to rounding.
    """
    decision = forage_rule(dmin, dmax, c, f, maxt, maxc)
    d = np.linspace(dmin, dmax, maxc)  # Death probability increases linearly
                                       # with individual condition.
    # We calculate forwards the number of individuals in each condition
    # by following the best strategy:
    for t in range(0, maxt):
        for i in range(1, maxc):
            if decision[i, t] == 0:  # Best strategy is to rest.
                numbers[i, t + 1] += numbers[i, t] * (1 - c)
                numbers[i - 1, t + 1] += numbers[i, t] * c
            if decision[i, t] == 1:  # Best strategy is to forage.
                numbers[i, t + 1] += numbers[i, t] * (1 - d[i]) * (1 - f)
                numbers[i + 1, t + 1] += numbers[i, t] * (1 - d[i]) * f
                numbers[0, t + 1] += numbers[i, t] * d[i]
        # Special case (row maxc, individuals in top condition cannot improve
        # their condition):
        if decision[maxc, t] == 0:  # Best strategy is to rest.
            numbers[maxc, t + 1] += numbers[maxc, t] * (1 - c)
            numbers[maxc - 1, t + 1] += numbers[maxc, t] * c
        if decision[maxc, t] == 1:  # Best strategy is to forage.
            numbers[maxc, t + 1] += numbers[maxc, t] * (1 - d[maxc - 1])
            numbers[0, t + 1] += numbers[maxc, t] * d[maxc - 1]
        numbers[0, t + 1] += numbers[0, t]  # Management of the dead row.
    return np.round(numbers)


def winter_simulation():
    """Fill a continuation of matrixes with individuals for several days.
    
    From a given number of individuals in each conditions at the first time 
    unit of the first day, this function uses the number matrixes to perform
    the one_day_simulation on several days, for a whole winter.
    
    Parameters
    ----------
    All parameters are asked in inputs.
    
    Returns
    -------
    The first output is the optimal strategy matrix, with blue cells when the 
    best strategy is to forage, and white cells when the best strategy is to 
    rest. Following outputs are the numbers matrixes recording the number of 
    individuals in each condition for each time unit day after day for the
    whole winter. The total number ofindividuals at the end of the simulation
    can not correspond to the total number of individuals at the beginning of
    the first day due to rounding.
    """
    # Parameters setting:
    dmin = float(input("Probability dmin of death per time unit if you are very lean (0 <= dmin <= 1): "))
    dmax = float(input("Probability dmax of death per time unit if you are very heavy (0 <= dmax <= 1 and dmax >= dmin): "))
    c = float(input("Rate c of consuming resources (0 <= c <= 1): "))
    f = float(input("Feeding efficiency f (0 <= f <= 1): "))
    maxt = int(input("Number maxt of time units the day is divided into (integer > 1): "))
    maxc = int(input("Number maxc of different condition units (integer >= 1): "))
    c_night = float(input("Probability c_night of consuming resources during the night (0 <= c_night <= 1): "))
    d_night = float(input("Probability d_night of death during the night (0 <= d_night <= 1): "))
    days = int(input("Number of days of the winter (integer >= 1): "))
    # Parameters validation:
    if dmin < 0 or dmin > 1:
        dmin = float(input("dmin have to be >= 0 and <= 1: "))
    if dmax < 0 or dmax > 1 or dmax < dmin:
        dmax = float(input("dmax have to be >= 0 and <= 1 and >= dmin: "))
    if c < 0 or c > 1:
        c = float(input("c have to be >= 0 and <= 1: "))
    if f < 0 or f > 1:
        f = float(input("f have to be >= 0 and <= 1: "))
    if maxt < 1:
        maxt = int(input("maxt have to be an integer > 1: "))
    if maxc < 1 :
        maxc = int(input("maxc have to be an integer >= 1: "))
    if d_night < 0 or d_night > 1:
        d_night = float(input("d_night have to be >= 0 and <= 1: "))
    if c_night < 0 or c_night > 1:
        c_night = float(input("c_night have to be >= 0 and <= 1: "))
    if days < 1:
        days = int(input("days have to be an integer >= 1: "))
    numbers = np.zeros([maxc + 1, maxt + 1])  # Initialization of the matrix.
    # Setting the initial numbers of individuals in each conditions:
    print("Condition 0 (Dead):")
    numbers[0, 0] = int(input("Number of individuals in this condition: "))
    for i in range(1, maxc):
        print(f"Condition {i}:")
        numbers[i, 0] = int(input("Number of individuals in this condition: "))
    print(f"Condition {maxc} (Top):")
    numbers[maxc, 0] = int(input("Number of individuals in this condition: "))   
    print(visualization(dmin, dmax, c, f, maxt, maxc))
    
    # Simulation for the whole winter:
    for i in range(0, days):
        numbers = one_day_simulation(dmin, dmax, c, f, maxt, maxc, numbers)
        print(f"Day {i + 1}:")
        print(np.round(numbers))
        new_numbers = np.zeros([maxc + 1, maxt + 1])  # Building of the matrix
                                                      # of the following day.
        new_numbers[0, 0] = numbers[0, maxt]          # This matrix starts from
                                                      # the numbers at the end
                                                      # of the previous day.
        for j in range(1, maxc + 1):
            new_numbers[j, 0] += numbers[j, maxt] * (1 - c_night) * \
                (1 - d_night)
            new_numbers[j - 1, 0] += numbers[j , maxt] * c_night * \
                (1 - d_night)
            new_numbers[0, 0] += numbers[j, maxt] * d_night
        numbers = new_numbers  # We go to the next day.


def multiply_matrix_list(l1, l2):
    """Multiply rows of two one-row matrixes.
    
    This function multiply the rows of two one-row matrixes which have the same
    dimension.

    Parameters
    ----------
    l1 : array
        One-row matrix.
    l2 : array
        One row matrix.
        
    Return
    -------
    The output is the l_mult matrix, result of the rows multiplication.
    """
    l_mult = np.array([l1[i] * l2[i] for i in range(len(l1))])
    return l_mult


def visualization(dmin, dmax, c, f, maxt, maxc):
    """Plot the optimal strategy matrix.
    
    Use the forage_rule function to plot the optimal strategy matrix.
    
    Parameters
    ----------
    dmin: float (between 0 and 1)
        probability of death per time unit if you are very lean.
    dmax: float (between 0 and 1)
        probability of death per time unit if you are very heavy.
    c: float (between 0 and 1)
        rate of consuming resources (= probability to decrease of one condition
                                     when resting).
    f: float (between 0 and 1)
        feeding efficiency (= probability to increase of one condition when
                            foraging).
    maxt: int > 0
        maximum time (number of time units the day is divided into).
    maxc: int > 0
        maximum condition (number of different condition units).

    Return
    -------
    The output is the optimal strategy matrix, with blue cells when the best
    strategy is to forage, and white cells when the best strategy is to rest.
    """
    # Building the grid:
    rule_grid = plt.matshow(np.delete(forage_rule(dmin, dmax, c, f, maxt,
                                                  maxc), maxt, axis = 1),
                            cmap = ListedColormap(['white', 'dodgerblue']))
    # Setting the caption:
    cbar = plt.colorbar(rule_grid)
    cbar.ax.get_yaxis().set_ticks([])
    # Setting the cells in the grid:
    for j, lab in enumerate(['Rest', 'Forage']):
        cbar.ax.text(0.35, (2 * j + 1) / 4.0, lab, ha='center', va='center',
                     rotation = 270)
        cbar.ax.get_yaxis().labelpad = 15
        cbar.ax.set_ylabel('Strategy', rotation = 270)
    plt.title('Matrix of strategy', pad = 15)
    plt.xlabel('Time', labelpad = 10)
    plt.ylabel('Condition', labelpad = 10)
    plt.hlines(y = np.arange(0, maxc) + 0.5, xmin = -0.5, xmax = maxt - 0.5,
               color = "black")
    plt.vlines(x = np.arange(0, maxt - 1) + 0.5, ymin = -0.5, 
               ymax = maxc + 0.5, color="black")
    plt.xticks(range(0, maxt), range(1, maxt + 1))
    plt.gca().xaxis.tick_bottom()
    plt.gca().invert_yaxis()  # We reverse the grid to have the dead
                              # individuals at the bottom of the grid and
                              # individuals in top condition at the top of the
                              # grid.
    plt.show()

In [ ]:
# Launch the simulation:
winter_simulation()